In [3]:
# 제시된 폴더를 순회하면서 shp파일의 'JIMOK' 칼럼이 '임야'이거나 'JIBUN' 칼럼의 마지막 값이 '임'으로 끝나는 행만 추출한다.
# 추출된 파일은 output_dir의 경로에 'SIDO_NM'+'SGG_NM'+'임야'.gpkg 형식으로 저장한다.
# 하나의 shp파일의 SIDO_NM, SGG_NM이 같은 경우 하나의 gpkg파일로 저장한다.

folder_dir = r"D:\Landslide\data\행정구역경계\2024년+03월+LX맵_필지_경상남도(구+국토정보기본도(필지))"
output_dir = r"D:\Landslide\data\행정구역경계\경남_필지_임야"

import os
import geopandas as gpd
import pandas as pd
from collections import defaultdict

# 출력 폴더 생성
os.makedirs(output_dir, exist_ok=True)

# SIDO_NM + SGG_NM 조합별로 데이터를 모으기 위한 딕셔너리
grouped_data = defaultdict(list)

# 전체 데이터를 합치기 위한 리스트
all_forest_data = []

# 폴더 내 모든 shp 파일 처리
for root, dirs, files in os.walk(folder_dir):
    for file in files:
        if file.endswith('.shp'):
            shp_path = os.path.join(root, file)
            print(f"Processing: {shp_path}")
            
            try:
                # shp 파일 읽기
                gdf = gpd.read_file(shp_path)
                
                # 필터링 조건: JIMOK이 '임야'이거나 JIBUN이 '임'으로 끝나는 경우
                filtered_gdf = gdf[
                    (gdf['JIMOK'] == '임야') | 
                    (gdf['JIBUN'].astype(str).str.endswith('임'))
                ]
                
                if not filtered_gdf.empty:
                    # 전체 데이터에 추가
                    all_forest_data.append(filtered_gdf)
                    
                    # SIDO_NM과 SGG_NM 조합별로 그룹화
                    for (sido_nm, sgg_nm), group in filtered_gdf.groupby(['SIDO_NM', 'SGG_NM']):
                        key = f"{sido_nm}{sgg_nm}"
                        grouped_data[key].append(group)
                        print(f"  - Found {len(group)} forest parcels in {sido_nm} {sgg_nm}")
                
            except Exception as e:
                print(f"Error processing {shp_path}: {e}")

# 그룹별로 파일 저장
for key, gdfs in grouped_data.items():
    if gdfs:
        # 같은 지역의 모든 데이터 합치기
        combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
        
        # 출력 파일명 생성
        output_filename = f"{key}임야.gpkg"
        output_path = os.path.join(output_dir, output_filename)
        
        # GPKG 파일로 저장
        combined_gdf.to_file(output_path, driver='GPKG')
        print(f"Saved: {output_path} ({len(combined_gdf)} parcels)")

# 모든 임야 데이터를 하나의 파일로 합치기
if all_forest_data:
    print("\nMerging all forest data into a single file...")
    merged_gdf = gpd.GeoDataFrame(pd.concat(all_forest_data, ignore_index=True))
    
    # 경상남도 전체 임야 필지 파일 저장
    merged_output_path = os.path.join(output_dir, "경상남도_임야_필지.gpkg")
    merged_gdf.to_file(merged_output_path, driver='GPKG')
    print(f"Saved merged file: {merged_output_path} ({len(merged_gdf)} total parcels)")

print(f"\nProcessing completed. Files saved to: {output_dir}")

Processing: D:\Landslide\data\행정구역경계\2024년+03월+LX맵_필지_경상남도(구+국토정보기본도(필지))\AL_48121_LAND_INFO_BASE_MAP_202403\48121.shp
  - Found 13291 forest parcels in 경상남도 창원시 의창구
Processing: D:\Landslide\data\행정구역경계\2024년+03월+LX맵_필지_경상남도(구+국토정보기본도(필지))\AL_48123_LAND_INFO_BASE_MAP_202403\48123.shp
  - Found 3792 forest parcels in 경상남도 창원시 성산구
Processing: D:\Landslide\data\행정구역경계\2024년+03월+LX맵_필지_경상남도(구+국토정보기본도(필지))\AL_48125_LAND_INFO_BASE_MAP_202403\48125.shp
  - Found 21390 forest parcels in 경상남도 창원시 마산합
Processing: D:\Landslide\data\행정구역경계\2024년+03월+LX맵_필지_경상남도(구+국토정보기본도(필지))\AL_48127_LAND_INFO_BASE_MAP_202403\48127.shp
  - Found 5000 forest parcels in 경상남도 창원시 마산회
Processing: D:\Landslide\data\행정구역경계\2024년+03월+LX맵_필지_경상남도(구+국토정보기본도(필지))\AL_48129_LAND_INFO_BASE_MAP_202403\48129.shp
  - Found 4952 forest parcels in 경상남도 창원시 진해구
Processing: D:\Landslide\data\행정구역경계\2024년+03월+LX맵_필지_경상남도(구+국토정보기본도(필지))\AL_48170_LAND_INFO_BASE_MAP_202403\48170.shp
  - Found 44665 forest parcels in 경상남도 진주시
Processing: